In [1]:
import numpy as np
import pandas as pd
from sklearn import *
import glob


In [2]:
teams = pd.read_csv('../../data/MTeams.csv')  
MTeam_spellings = pd.read_csv('../../data/MTeamspellings.csv', encoding='ISO-8859-1')  
WTeam_spellings = pd.read_csv('../../data/WTeamspellings.csv', encoding='ISO-8859-1')  

In [3]:
MTeam_spellings.head()

,TeamNameSpelling,TeamID
0,a&m-corpus chris,1394
1,a&m-corpus christi,1394
2,abilene chr,1101
3,abilene christian,1101
4,abilene-christian,1101


In [4]:
teams_spelling = pd.concat([MTeam_spellings, WTeam_spellings])
teams_spelling = teams_spelling.groupby(by='TeamID', as_index=False)['TeamNameSpelling'].count()
teams_spelling.columns = ['TeamID', 'TeamNameCount']
teams = pd.merge(teams, teams_spelling, how='left', on=['TeamID'])
del teams_spelling

In [5]:
teams.head()

,TeamID,TeamName,FirstD1Season,LastD1Season,TeamNameCount
0,1101,Abilene Chr,2014,2025,3
1,1102,Air Force,1985,2025,2
2,1103,Akron,1985,2025,1
3,1104,Alabama,1985,2025,1
4,1105,Alabama A&M,2000,2025,2


In [6]:
MRegularSeasonCompactResults = pd.read_csv('../../data/MRegularSeasonCompactResults.csv')
WRegularSeasonCompactResults = pd.read_csv('../../data/WRegularSeasonCompactResults.csv')

In [7]:
MRegularSeasonDetailedResults = pd.read_csv('../../data/MRegularSeasonDetailedResults.csv')
WRegularSeasonDetailedResults = pd.read_csv('../../data/WRegularSeasonDetailedResults.csv')

In [8]:
MNCAATourneyCompactResults = pd.read_csv('../../data/MNCAATourneyCompactResults.csv')
WNCAATourneyCompactResults = pd.read_csv('../../data/WNCAATourneyCompactResults.csv')

In [9]:
MNCAATourneyDetailedResults = pd.read_csv('../../data/MNCAATourneyDetailedResults.csv') 
WNCAATourneyDetailedResults = pd.read_csv('../../data/WNCAATourneyDetailedResults.csv')

In [10]:
MNCAATourneySeeds = pd.read_csv('../../data/MNCAATourneySeeds.csv')
WNCAATourneySeeds = pd.read_csv('../../data/WNCAATourneySeeds.csv')

In [11]:
MGameCities = pd.read_csv('../../data/MGameCities.csv') 
WGameCities = pd.read_csv('../../data/WGameCities.csv')

In [12]:
MSeasons = pd.read_csv('../../data/MSeasons.csv') 
WSeasons = pd.read_csv('../../data/WSeasons.csv')

In [13]:
season_cresults = pd.concat([MRegularSeasonCompactResults, WRegularSeasonCompactResults])
season_dresults = pd.concat([MRegularSeasonDetailedResults, WRegularSeasonDetailedResults])
tourney_cresults = pd.concat([MNCAATourneyCompactResults, WNCAATourneyCompactResults])
tourney_dresults = pd.concat([MNCAATourneyDetailedResults, WNCAATourneyDetailedResults])
seeds = pd.concat([MNCAATourneySeeds, WNCAATourneySeeds])
gcities = pd.concat([MGameCities, WGameCities])
seasons = pd.concat([MSeasons, WSeasons])

seeds = {'_'.join(map(str,[int(k1),k2])):int(v[1:3]) for k1, v, k2 in seeds[['Season', 'Seed', 'TeamID']].values}
cities = pd.read_csv('../../data/cities.csv') 
sub = pd.read_csv('../../data/SampleSubmissionStage2.csv')

In [14]:
season_cresults['ST'] = 'S'
season_dresults['ST'] = 'S'
tourney_cresults['ST'] = 'T'
tourney_dresults['ST'] = 'T'
games = pd.concat((season_cresults, tourney_cresults), axis=0, ignore_index=True)
games = pd.concat((season_dresults, tourney_dresults), axis=0, ignore_index=True)

In [15]:
games.columns

Index(['Season', 'DayNum', 'WTeamID', 'WScore', 'LTeamID', 'LScore', 'WLoc',
       'NumOT', 'WFGM', 'WFGA', 'WFGM3', 'WFGA3', 'WFTM', 'WFTA', 'WOR', 'WDR',
       'WAst', 'WTO', 'WStl', 'WBlk', 'WPF', 'LFGM', 'LFGA', 'LFGM3', 'LFGA3',
       'LFTM', 'LFTA', 'LOR', 'LDR', 'LAst', 'LTO', 'LStl', 'LBlk', 'LPF',
       'ST'],
      dtype='object')

In [16]:
games.head()

,Season,DayNum,WTeamID,WScore,LTeamID,LScore,WLoc,NumOT,WFGM,WFGA,...,LFTM,LFTA,LOR,LDR,LAst,LTO,LStl,LBlk,LPF,ST
0,2003,10,1104,68,1328,62,N,0,27,58,...,16,22,10,22,8,18,9,2,20,S
1,2003,10,1272,70,1393,63,N,0,26,62,...,9,20,20,25,7,12,8,6,16,S
2,2003,11,1266,73,1437,61,N,0,24,58,...,14,23,31,22,9,12,2,5,23,S
3,2003,11,1296,56,1457,50,N,0,18,38,...,8,15,17,20,9,19,4,3,23,S
4,2003,11,1400,77,1208,71,N,0,30,61,...,17,27,21,15,12,10,7,1,14,S


In [17]:
games.reset_index(drop=True, inplace=True)
games['WLoc'] = games['WLoc'].map({'A': 1, 'H': 2, 'N': 3})

games['ID'] = games.apply(lambda r: '_'.join(map(str, [r['Season']]+sorted([r['WTeamID'],r['LTeamID']]))), axis=1)
games['IDTeams'] = games.apply(lambda r: '_'.join(map(str, sorted([r['WTeamID'],r['LTeamID']]))), axis=1)
games['Team1'] = games.apply(lambda r: sorted([r['WTeamID'],r['LTeamID']])[0], axis=1)
games['Team2'] = games.apply(lambda r: sorted([r['WTeamID'],r['LTeamID']])[1], axis=1)
games['IDTeam1'] = games.apply(lambda r: '_'.join(map(str, [r['Season'], r['Team1']])), axis=1)
games['IDTeam2'] = games.apply(lambda r: '_'.join(map(str, [r['Season'], r['Team2']])), axis=1)

games['Team1Seed'] = games['IDTeam1'].map(seeds).fillna(0)
games['Team2Seed'] = games['IDTeam2'].map(seeds).fillna(0)

games['ScoreDiff'] = games['WScore'] - games['LScore']
games['Pred'] = games.apply(lambda r: 1. if sorted([r['WTeamID'],r['LTeamID']])[0]==r['WTeamID'] else 0., axis=1)
games['ScoreDiffNorm'] = games.apply(lambda r: r['ScoreDiff'] * -1 if r['Pred'] == 0. else r['ScoreDiff'], axis=1)
games['SeedDiff'] = games['Team1Seed'] - games['Team2Seed'] 
games = games.fillna(-1)

In [18]:
# Winning Team's Performance
games['WFG%'] = games['WFGM'] / games['WFGA']
games['WFGM3%'] = games['WFGM3'] / games['WFGA3']
games['WFT%'] = games['WFTM'] / games['WFTA']
games['WTO%'] = games['WTO'] / (games['WFGM'] + games['WFGA'] + games['WFTM'] + games['WFTA'])
games['WOffEfficiency'] = games['WAst'] / (games['WFGM'] + games['WFGA'] + games['WFTM'] + games['WFTA'])

# Losing Team's Performance
games['LFG%'] = games['LFGM'] / games['LFGA']
games['LFGM3%'] = games['LFGM3'] / games['LFGA3']
games['LFT%'] = games['LFTM'] / games['LFTA']
games['LTO%'] = games['LTO'] / (games['LFGM'] + games['LFGA'] + games['LFTM'] + games['LFTA'])
games['LOffEfficiency'] = games['LAst'] / (games['LFGM'] + games['LFGA'] + games['LFTM'] + games['LFTA'])

# Game-Level Metrics
games['ScoreDiff'] = games['WScore'] - games['LScore']
games['ScoreDiffNorm'] = games['ScoreDiff'] / (games['WScore'] + games['LScore'])
games['SeedDiff'] = abs(games['Team1Seed'] - games['Team2Seed'])

# Winning Team's Rebound Differential
games['WRebDiff'] = (games['WOR'] + games['WDR']) - (games['LOR'] + games['LDR'])

games.head()


,Season,DayNum,WTeamID,WScore,LTeamID,LScore,WLoc,NumOT,WFGM,WFGA,...,WFGM3%,WFT%,WTO%,WOffEfficiency,LFG%,LFGM3%,LFT%,LTO%,LOffEfficiency,WRebDiff
0,2003,10,1104,68,1328,62,3,0,27,58,...,0.214286,0.611111,0.201754,0.114035,0.415094,0.200000,0.727273,0.159292,0.070796,6
1,2003,10,1272,70,1393,63,3,0,26,62,...,0.400000,0.526316,0.111111,0.136752,0.358209,0.250000,0.450000,0.100000,0.058333,-2
2,2003,11,1266,73,1437,61,3,0,24,58,...,0.444444,0.586207,0.078125,0.117188,0.301370,0.115385,0.608696,0.090909,0.068182,-10
3,2003,11,1296,56,1457,50,3,0,18,38,...,0.333333,0.548387,0.115385,0.105769,0.367347,0.272727,0.533333,0.211111,0.100000,-12
4,2003,11,1400,77,1208,71,3,0,30,61,...,0.428571,0.846154,0.121739,0.104348,0.387097,0.375000,0.629630,0.076923,0.092308,3


In [19]:
games.columns

Index(['Season', 'DayNum', 'WTeamID', 'WScore', 'LTeamID', 'LScore', 'WLoc',
       'NumOT', 'WFGM', 'WFGA', 'WFGM3', 'WFGA3', 'WFTM', 'WFTA', 'WOR', 'WDR',
       'WAst', 'WTO', 'WStl', 'WBlk', 'WPF', 'LFGM', 'LFGA', 'LFGM3', 'LFGA3',
       'LFTM', 'LFTA', 'LOR', 'LDR', 'LAst', 'LTO', 'LStl', 'LBlk', 'LPF',
       'ST', 'ID', 'IDTeams', 'Team1', 'Team2', 'IDTeam1', 'IDTeam2',
       'Team1Seed', 'Team2Seed', 'ScoreDiff', 'Pred', 'ScoreDiffNorm',
       'SeedDiff', 'WFG%', 'WFGM3%', 'WFT%', 'WTO%', 'WOffEfficiency', 'LFG%',
       'LFGM3%', 'LFT%', 'LTO%', 'LOffEfficiency', 'WRebDiff'],
      dtype='object')

In [20]:
games.head()

,Season,DayNum,WTeamID,WScore,LTeamID,LScore,WLoc,NumOT,WFGM,WFGA,...,WFGM3%,WFT%,WTO%,WOffEfficiency,LFG%,LFGM3%,LFT%,LTO%,LOffEfficiency,WRebDiff
0,2003,10,1104,68,1328,62,3,0,27,58,...,0.214286,0.611111,0.201754,0.114035,0.415094,0.200000,0.727273,0.159292,0.070796,6
1,2003,10,1272,70,1393,63,3,0,26,62,...,0.400000,0.526316,0.111111,0.136752,0.358209,0.250000,0.450000,0.100000,0.058333,-2
2,2003,11,1266,73,1437,61,3,0,24,58,...,0.444444,0.586207,0.078125,0.117188,0.301370,0.115385,0.608696,0.090909,0.068182,-10
3,2003,11,1296,56,1457,50,3,0,18,38,...,0.333333,0.548387,0.115385,0.105769,0.367347,0.272727,0.533333,0.211111,0.100000,-12
4,2003,11,1400,77,1208,71,3,0,30,61,...,0.428571,0.846154,0.121739,0.104348,0.387097,0.375000,0.629630,0.076923,0.092308,3


In [21]:
c_score_col = ['NumOT', 'WFGM', 'WFGA', 'WFGM3', 'WFGA3', 'WFTM', 'WFTA', 'WOR', 'WDR', 'WAst', 'WTO', 'WStl',
 'WBlk', 'WPF', 'LFGM', 'LFGA', 'LFGM3', 'LFGA3', 'LFTM', 'LFTA', 'LOR', 'LDR', 'LAst', 'LTO', 'LStl',
 'LBlk', 'LPF']
c_score_agg = ['sum', 'mean', 'median', 'max', 'min', 'std', 'skew', 'nunique']
gb = games.groupby(by=['IDTeams']).agg({k: c_score_agg for k in c_score_col}).reset_index()
gb.columns = [''.join(c) + '_c_score' for c in gb.columns]

games = games[games['ST']=='T']

In [22]:
games.head()

,Season,DayNum,WTeamID,WScore,LTeamID,LScore,WLoc,NumOT,WFGM,WFGA,...,WFGM3%,WFT%,WTO%,WOffEfficiency,LFG%,LFGM3%,LFT%,LTO%,LOffEfficiency,WRebDiff
198374,2003,134,1421,92,1411,84,3,1,32,69,...,0.379310,0.653846,0.083333,0.118056,0.432836,0.387097,0.451613,0.106383,0.113475,-1
198375,2003,136,1112,80,1436,51,3,0,31,66,...,0.304348,0.785714,0.131148,0.180328,0.312500,0.250000,1.000000,0.173469,0.122449,13
198376,2003,136,1113,84,1272,71,3,0,31,59,...,0.428571,0.727273,0.070312,0.140625,0.362319,0.250000,0.666667,0.093023,0.085271,-5
198377,2003,136,1141,79,1166,73,3,0,29,53,...,0.428571,0.720000,0.144000,0.120000,0.450000,0.411765,0.705882,0.181034,0.172414,0
198378,2003,136,1143,76,1301,74,3,1,27,64,...,0.350000,0.652174,0.100775,0.131783,0.446429,0.428571,0.750000,0.120690,0.137931,2


In [23]:
sub['WLoc'] = 3
sub['Season'] = sub['ID'].map(lambda x: x.split('_')[0])
sub['Season'] = sub['ID'].map(lambda x: x.split('_')[0])
sub['Season'] = sub['Season'].astype(int)
sub['Team1'] = sub['ID'].map(lambda x: x.split('_')[1])
sub['Team2'] = sub['ID'].map(lambda x: x.split('_')[2])
sub['IDTeams'] = sub.apply(lambda r: '_'.join(map(str, [r['Team1'], r['Team2']])), axis=1)
sub['IDTeam1'] = sub.apply(lambda r: '_'.join(map(str, [r['Season'], r['Team1']])), axis=1)
sub['IDTeam2'] = sub.apply(lambda r: '_'.join(map(str, [r['Season'], r['Team2']])), axis=1)
sub['Team1Seed'] = sub['IDTeam1'].map(seeds).fillna(0)
sub['Team2Seed'] = sub['IDTeam2'].map(seeds).fillna(0)
sub['SeedDiff'] = sub['Team1Seed'] - sub['Team2Seed'] 
sub = sub.fillna(-1)

In [25]:
sub.head()

,ID,Pred,WLoc,Season,Team1,Team2,IDTeams,IDTeam1,IDTeam2,Team1Seed,Team2Seed,SeedDiff
0,2025_1101_1102,0.5,3,2025,1101,1102,1101_1102,2025_1101,2025_1102,0.0,0.0,0.0
1,2025_1101_1103,0.5,3,2025,1101,1103,1101_1103,2025_1101,2025_1103,0.0,0.0,0.0
2,2025_1101_1104,0.5,3,2025,1101,1104,1101_1104,2025_1101,2025_1104,0.0,0.0,0.0
3,2025_1101_1105,0.5,3,2025,1101,1105,1101_1105,2025_1101,2025_1105,0.0,0.0,0.0
4,2025_1101_1106,0.5,3,2025,1101,1106,1101_1106,2025_1101,2025_1106,0.0,0.0,0.0


In [24]:
# Winning Team's Performance
sub['WFG%'] = sub['WFGM'] / sub['WFGA']
sub['WFGM3%'] = sub['WFGM3'] / sub['WFGA3']
sub['WFT%'] = sub['WFTM'] / sub['WFTA']
sub['WTO%'] = sub['WTO'] / (sub['WFGM'] + sub['WFGA'] + sub['WFTM'] + sub['WFTA'])
sub['WOffEfficiency'] = sub['WAst'] / (sub['WFGM'] + sub['WFGA'] + sub['WFTM'] + sub['WFTA'])

# Losing Team's Performance
sub['LFG%'] = sub['LFGM'] / sub['LFGA']
sub['LFGM3%'] = sub['LFGM3'] / sub['LFGA3']
sub['LFT%'] = sub['LFTM'] / sub['LFTA']
sub['LTO%'] = sub['LTO'] / (sub['LFGM'] + sub['LFGA'] + sub['LFTM'] + sub['LFTA'])
sub['LOffEfficiency'] = sub['LAst'] / (sub['LFGM'] + sub['LFGA'] + sub['LFTM'] + sub['LFTA'])

# Game-Level Metrics
sub['ScoreDiff'] = sub['WScore'] - sub['LScore']
sub['ScoreDiffNorm'] = sub['ScoreDiff'] / (sub['WScore'] + sub['LScore'])
sub['SeedDiff'] = abs(sub['Team1Seed'] - sub['Team2Seed'])

# Winning Team's Rebound Differential
sub['WRebDiff'] = (sub['WOR'] + sub['WDR']) - (sub['LOR'] + sub['LDR'])


KeyError: 'WFGM'

In [ ]:
sub.head()

,ID,Pred,WLoc,Season,Team1,Team2,IDTeams,IDTeam1,IDTeam2,Team1Seed,Team2Seed,SeedDiff
0,2025_1101_1102,0.5,3,2025,1101,1102,1101_1102,2025_1101,2025_1102,0.0,0.0,0.0
1,2025_1101_1103,0.5,3,2025,1101,1103,1101_1103,2025_1101,2025_1103,0.0,0.0,0.0
2,2025_1101_1104,0.5,3,2025,1101,1104,1101_1104,2025_1101,2025_1104,0.0,0.0,0.0
3,2025_1101_1105,0.5,3,2025,1101,1105,1101_1105,2025_1101,2025_1105,0.0,0.0,0.0
4,2025_1101_1106,0.5,3,2025,1101,1106,1101_1106,2025_1101,2025_1106,0.0,0.0,0.0


In [26]:
games = pd.merge(games, gb, how='left', left_on='IDTeams', right_on='IDTeams_c_score')
sub = pd.merge(sub, gb, how='left', left_on='IDTeams', right_on='IDTeams_c_score')

col = [c for c in games.columns if c not in ['ID', 'DayNum', 'ST', 'Team1', 'Team2', 'IDTeams', 'IDTeam1', 'IDTeam2', 'WTeamID', 'WScore', 'LTeamID', 'LScore', 'NumOT', 'Pred', 'ScoreDiff', 'ScoreDiffNorm', 'WLoc'] + c_score_col]

In [27]:
games.head()


,Season,DayNum,WTeamID,WScore,LTeamID,LScore,WLoc,NumOT,WFGM,WFGA,...,LBlkskew_c_score,LBlknunique_c_score,LPFsum_c_score,LPFmean_c_score,LPFmedian_c_score,LPFmax_c_score,LPFmin_c_score,LPFstd_c_score,LPFskew_c_score,LPFnunique_c_score
0,2003,134,1421,92,1411,84,3,1,32,69,...,NaN,1,22,22.0,22.0,22,22,NaN,NaN,1
1,2003,136,1112,80,1436,51,3,0,31,66,...,NaN,1,15,15.0,15.0,15,15,NaN,NaN,1
2,2003,136,1113,84,1272,71,3,0,31,59,...,NaN,1,18,18.0,18.0,18,18,NaN,NaN,1
3,2003,136,1141,79,1166,73,3,0,29,53,...,NaN,2,35,17.5,17.5,21,14,4.949747,NaN,2
4,2003,136,1143,76,1301,74,3,1,27,64,...,-0.422521,4,66,16.5,17.0,19,13,2.645751,-0.863919,4


In [ ]:
sub.head()

,ID,Pred,WLoc,Season,Team1,Team2,IDTeams,IDTeam1,IDTeam2,Team1Seed,...,LBlkskew_c_score,LBlknunique_c_score,LPFsum_c_score,LPFmean_c_score,LPFmedian_c_score,LPFmax_c_score,LPFmin_c_score,LPFstd_c_score,LPFskew_c_score,LPFnunique_c_score
0,2025_1101_1102,0.5,3,2025,1101,1102,1101_1102,2025_1101,2025_1102,0.0,...,NaN,1.0,14.0,14.0,14.0,14.0,14.0,NaN,NaN,1.0
1,2025_1101_1103,0.5,3,2025,1101,1103,1101_1103,2025_1101,2025_1103,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2025_1101_1104,0.5,3,2025,1101,1104,1101_1104,2025_1101,2025_1104,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2025_1101_1105,0.5,3,2025,1101,1105,1101_1105,2025_1101,2025_1105,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2025_1101_1106,0.5,3,2025,1101,1106,1101_1106,2025_1101,2025_1106,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
import optuna
from sklearn import ensemble
from sklearn.metrics import *
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer

/Users/danchizik/Desktop/marchMadness/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
imputer = SimpleImputer(strategy='mean')  
scaler = StandardScaler()

In [ ]:
X = games[col].fillna(-1)
X_imputed = imputer.fit_transform(X)
X_scaled = scaler.fit_transform(X_imputed)

y = games['Pred']

In [ ]:
import torch
from torch.utils.data import TensorDataset, DataLoader
from torch.utils.data import random_split

In [ ]:
# turn numpy arrays into tensors
X_train_tensor = torch.tensor(X_scaled, dtype=torch.float32)  # Convert features to float
y_train_tensor = torch.tensor(y, dtype=torch.float32)  # Convert labels to long (for classification)
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)

# Train, valid split
train_size = int(0.8 * len(train_dataset))  # 80% for training
val_size = len(train_dataset) - train_size  # Remaining 20% for validation
train_dataset, val_dataset = random_split(train_dataset, [train_size, val_size])


batch_size = 32  # Set batch size
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

In [ ]:
from torch import nn

In [ ]:
class NeuralNetwork(nn.Module):
    def __init__(self, d_in, d_out, d_hidden, n_layers = 2):
        super().__init__()
        layers = [nn.Linear(d_in, d_hidden), nn.BatchNorm1d(d_hidden),
            nn.ReLU()]
        for layer in range(n_layers):
            layers += [nn.Linear(d_hidden, d_hidden), nn.BatchNorm1d(d_hidden),nn.ReLU(), nn.Dropout(p=0.3)]
        layers += [nn.Linear(d_hidden, d_out)]
        self.linear_relu_stack = nn.Sequential(*layers)

    def forward(self, x):
        logits = self.linear_relu_stack(x)
        return logits


In [ ]:
import torch.optim as optim

In [ ]:
model = NeuralNetwork(X_scaled.shape[1], 1, 100)
print(model)
loss_fn = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.01)  # Adjust learning rate as needed

# Training function
def train(model, train_loader, loss_fn, optimizer, device):
    model.train()  # Set model to training mode
    total_loss = 0
    
    for batch_X, batch_y in train_loader:
        batch_X, batch_y = batch_X.to(device), batch_y.to(device)  # Move data to GPU if available

        optimizer.zero_grad()  # Reset gradients
        outputs = model(batch_X).squeeze()  # Forward pass (ensure output shape matches labels)
        loss = loss_fn(outputs, batch_y)  # Compute MSE loss
        loss.backward()  # Backpropagation
        optimizer.step()  # Update weights
        
        total_loss += loss.item()  # Accumulate loss
    
    return total_loss / len(train_loader)  # Return average loss per batch

# Evaluation function
def evaluate(model, val_loader, loss_fn, device):
    model.eval()  # Set model to evaluation mode
    total_loss = 0
    
    with torch.no_grad():  # Disable gradient computation
        for batch_X, batch_y in val_loader:
            batch_X, batch_y = batch_X.to(device), batch_y.to(device)
            outputs = model(batch_X).squeeze()  # Ensure output shape matches labels
            
            loss = loss_fn(outputs, batch_y)
            total_loss += loss.item()

    avg_loss = total_loss / len(val_loader)
    return avg_loss  # Return MSE loss (lower is better)

NeuralNetwork(
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=232, out_features=100, bias=True)
    (1): BatchNorm1d(100, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): Linear(in_features=100, out_features=100, bias=True)
    (4): BatchNorm1d(100, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): ReLU()
    (6): Dropout(p=0.3, inplace=False)
    (7): Linear(in_features=100, out_features=100, bias=True)
    (8): BatchNorm1d(100, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (9): ReLU()
    (10): Dropout(p=0.3, inplace=False)
    (11): Linear(in_features=100, out_features=1, bias=True)
  )
)


In [ ]:
# Move model to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Training loop
num_epochs = 20  # Adjust the number of epochs based on performance

for epoch in range(num_epochs):
    train_loss = train(model, train_loader, loss_fn, optimizer, device)
    val_loss = evaluate(model, val_loader, loss_fn, device)
    
    print(f"Epoch {epoch+1}/{num_epochs}:")
    print(f"  Train Loss (MSE): {train_loss:.4f}")
    print(f"  Validation Loss (MSE): {val_loss:.4f}")

Epoch 1/20:
  Train Loss (MSE): 0.3260
  Validation Loss (MSE): 0.2397
Epoch 2/20:
  Train Loss (MSE): 0.2527
  Validation Loss (MSE): 0.2754
Epoch 3/20:
  Train Loss (MSE): 0.2260
  Validation Loss (MSE): 0.2207
Epoch 4/20:
  Train Loss (MSE): 0.1997
  Validation Loss (MSE): 0.2001
Epoch 5/20:
  Train Loss (MSE): 0.1883
  Validation Loss (MSE): 0.1907
Epoch 6/20:
  Train Loss (MSE): 0.1756
  Validation Loss (MSE): 0.1983
Epoch 7/20:
  Train Loss (MSE): 0.1727
  Validation Loss (MSE): 0.2065
Epoch 8/20:
  Train Loss (MSE): 0.1686
  Validation Loss (MSE): 0.1891
Epoch 9/20:
  Train Loss (MSE): 0.1679
  Validation Loss (MSE): 0.2147
Epoch 10/20:
  Train Loss (MSE): 0.1619
  Validation Loss (MSE): 0.2077
Epoch 11/20:
  Train Loss (MSE): 0.1576
  Validation Loss (MSE): 0.2168
Epoch 12/20:
  Train Loss (MSE): 0.1546
  Validation Loss (MSE): 0.2010
Epoch 13/20:
  Train Loss (MSE): 0.1485
  Validation Loss (MSE): 0.1969
Epoch 14/20:
  Train Loss (MSE): 0.1479
  Validation Loss (MSE): 0.2210
E

In [ ]:
X_submit = sub[col].fillna(-1)
X_submit_imputed = imputer.transform(X_submit)
X_submit_scaled = scaler.transform(X_submit_imputed)


X_submit_tensor = torch.tensor(X_submit_scaled, dtype=torch.float32)

# Move to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
X_submit_tensor = X_submit_tensor.to(device)

# Set model to evaluation mode
model.eval()

# Make predictions (disable gradients for efficiency)
with torch.no_grad():
    y_preds = model(X_submit_tensor).cpu().numpy()  # Move back to CPU for saving
        

KeyError: "['WFG%', 'WFGM3%', 'WFT%', 'WTO%', 'WOffEfficiency', 'LFG%', 'LFGM3%', 'LFT%', 'LTO%', 'LOffEfficiency', 'WRebDiff'] not in index"

In [ ]:
sub['Pred'] = y_preds
sub.head()
# sub[['ID', 'Pred']].to_csv('submission.csv', index=False)

,ID,Pred,WLoc,Season,Team1,Team2,IDTeams,IDTeam1,IDTeam2,Team1Seed,...,LBlkskew_c_score,LBlknunique_c_score,LPFsum_c_score,LPFmean_c_score,LPFmedian_c_score,LPFmax_c_score,LPFmin_c_score,LPFstd_c_score,LPFskew_c_score,LPFnunique_c_score
0,2025_1101_1102,0.402358,3,2025,1101,1102,1101_1102,2025_1101,2025_1102,0.0,...,NaN,1.0,14.0,14.0,14.0,14.0,14.0,NaN,NaN,1.0
1,2025_1101_1103,0.309249,3,2025,1101,1103,1101_1103,2025_1101,2025_1103,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2025_1101_1104,0.309249,3,2025,1101,1104,1101_1104,2025_1101,2025_1104,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2025_1101_1105,0.309249,3,2025,1101,1105,1101_1105,2025_1101,2025_1105,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2025_1101_1106,0.309249,3,2025,1101,1106,1101_1106,2025_1101,2025_1106,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
sub[['ID', 'Pred']].head(20)

,ID,Pred
0,2025_1101_1102,0.402358
1,2025_1101_1103,0.309249
2,2025_1101_1104,0.309249
3,2025_1101_1105,0.309249
4,2025_1101_1106,0.309249
5,2025_1101_1107,0.309249
6,2025_1101_1108,0.309249
7,2025_1101_1110,0.309249
8,2025_1101_1111,0.309249
9,2025_1101_1112,0.309249


In [ ]:
teams = pd.read_csv('../../data/MTeams.csv')  
teams.head(20)

,TeamID,TeamName,FirstD1Season,LastD1Season
0,1101,Abilene Chr,2014,2025
1,1102,Air Force,1985,2025
2,1103,Akron,1985,2025
3,1104,Alabama,1985,2025
4,1105,Alabama A&M,2000,2025
5,1106,Alabama St,1985,2025
6,1107,SUNY Albany,2000,2025
7,1108,Alcorn St,1985,2025
8,1109,Alliant Intl,1985,1991
9,1110,American Univ,1985,2025
